In [ ]:
# Day3 Project Challenge - COVID module

Find the original Notebook with detailed instructions by **Daniel Pass** [here](https://github.com/passdan/PracticalPythonProgrammingForBiologists/blob/main/Day4/P34B-Day4-Project-covid_modules.ipynb).

Find the input file [here](inputData/Day4-Project-nextclade.fasta) and [here](inputData/Day4-Project-nextclade_metadata.csv).

> In this exercise we are going to look at a complex dataset (n=126) of covid S protein reference sequences and associated metadata and statistics, and we're going to use some 3rd party modules to perform some higher-level analysis.
> 
> As we have seen today there are many (many!) ways to approach any dataset. First consider a question that you want to test from this data. If you are more interested in statistical testing then you can focus on the metadata and statistics with pandas/scipy. If you are more interested in sequence bioinformatics then you can work with the fasta data. However you'll likely need both to fully investigate.
>
> You have been provided with two files:
>
> * [<code>Day4-Project-nextclade_metadata.csv</code>](inputData/Day4-Project-nextclade_metadata.csv) - Data & Metadata of 126 reference covid19 strains and the Spike (S) protein modificiation data
> * [<code>Day4-Project-nextclade.fasta</code>](inputData/Day4-Project-nextclade.fasta) - Reference fasta sequence file for the corresponding spike proteins

In [16]:
import pandas as pd
from Bio import SeqIO, Seq


# Read in and filter the metadata file to remove bad and missing data
covid_data = pd.read_csv("inputData/Day4-Project-nextclade_metadata.csv")

covid_data_cleaned = covid_data.dropna()

# Read in the fasta sequence file. Only include sequences that have passed your step one filter!
covid_seqs = []

#for seq in SeqIO.parse("inputData/Day4-Project-nextclade.fasta", "fasta"):

print("OX458355" in covid_data_cleaned["seqName"])


ModuleNotFoundError: No module named 'Bio'